# Join on Census data

Using tidycensus, this notebook grabs zip-code level demographic information and adds to the aggregated school-level data.

## Setup Python and R environment
you can ignore this section

In [2]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [3]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
%%R

# My commonly used R imports

require('tidyverse')
require(data.table)


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse
Loading required package: data.table
data.table 1.17.0 using 4 threads (see ?getDTthreads).  Latest news: r-datatable.com

Attaching package: ‘data.table’

The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose



## Load & Clean Data

👉 Load the data along with the census connectors below (the output of the `connect-to-census.ipynb` notebook) and do any cleanup you'd like to do.

In [5]:
%%R

df <- fread("../data/2021-22-crdc-data/merged_data.csv")

|--------------------------------------------------|
|==================================================|


In [6]:
%%R

df %>% head(5)

   LEA_STATE_x LEA_STATE_NAME_x  LEAID             LEA_NAME_x SCHID
      <char>           <char> <char>                 <char> <int>
          AL          ALABAMA 100002 Alabama Youth Services 99995
          AL          ALABAMA 100005       Albertville City   870
          AL          ALABAMA 100005       Albertville City   871
          AL          ALABAMA 100005       Albertville City   879
          AL          ALABAMA 100005       Albertville City   889
                          SCH_NAME    COMBOKEY     JJ SCH_DISCWODIS_REF_HI_M
                          <char>      <char> <char>                  <int>
                  AUTAUGA CAMPUS 10000299995    Yes                     -9
       Albertville Middle School 10000500870     No                      0
         Albertville High School 10000500871     No                      0
 Albertville Intermediate School 10000500879     No                      0
   Albertville Elementary School 10000500889     No                      0
   SCH_DI

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)




                   -9                   -9                   -9
                   -9                   -9                   -9
                   90                   59                    0
                   -9                   -9                   -9
                   -9                   -9                   -9
   SCH_SCIENR_PHYS_AM_F SCH_SCIENR_PHYS_AS_M SCH_SCIENR_PHYS_AS_F
                <num>                <num>                <num>
                   -9                   -9                   -9
                   -9                   -9                   -9
                    0                    0                    0
                   -9                   -9                   -9
                   -9                   -9                   -9
   SCH_SCIENR_PHYS_HP_M SCH_SCIENR_PHYS_HP_F SCH_SCIENR_PHYS_BL_M
                <num>                <num>                <num>
                   -9                   -9                   -9
                   -9              

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



## 👉 Grab Census Data

1. loading the Census API key

In [7]:
import dotenv

# Load the environment variables
# (loads CENSUS_API_KEY from .env)
dotenv.load_dotenv()


True

In [8]:
%%R 

require('tidycensus')

# because it an environment variable, we don't have to 
# explicitly pass this string to R, it is readable here
# in this R cell.
census_api_key(Sys.getenv("CENSUS_API_KEY"))

Loading required package: tidycensus
To install your API key for use in future sessions, run this function with `install = TRUE`.


2. Decide which Census variables you want

    Use <https://censusreporter.org/> to figure out which tables you want. (if censusreporter is down, check out the code in the cell below)

    -   Scroll to the bottom of the page to see the tables.
    -   If you already know the table ID, stick that in the "Explore" section to learn more about that table.

    By default this code loads (B01003_001) which we found in censusreporter here: https://censusreporter.org/tables/B01003/

    - find some other variables that you're also interested in
    - don't forget to pick a geography like "tract", "county" or "block group". here is the list of [all geographies](https://walker-data.com/tidycensus/articles/basic-usage.html#geography-in-tidycensus
    ).


In [9]:
%%R 

# Finding the Census Varaibles for the ACS 5 year survey
# Generall you'd do this in CensusReporter, but since it's down sometimes, here it is using tidycensus's load_variables function

# get every single variable in the ACS5
all_census_vars <- load_variables(2021, "acs5", cache = TRUE) 

filtered_census_vars <- all_census_vars %>% 
    filter(grepl("income", label, ignore.case = TRUE))   # filter to those containing "median income"
    
# write to CSV so we can view it in python
filtered_census_vars %>% 
    write_csv("../data/filtered_census_vars.csv")

# show the first few rows
filtered_census_vars %>%
    select(-geography) %>% # remove the geography column
    print(n = 20) # print the first 20 rows

# A tibble: 2,803 × 3
   name         label                                                    concept
   <chr>        <chr>                                                    <chr>  
 1 B06010PR_002 Estimate!!Total:!!No income                              PLACE …
 2 B06010PR_003 Estimate!!Total:!!With income:                           PLACE …
 3 B06010PR_004 Estimate!!Total:!!With income:!!$1 to $9,999 or loss     PLACE …
 4 B06010PR_005 Estimate!!Total:!!With income:!!$10,000 to $14,999       PLACE …
 5 B06010PR_006 Estimate!!Total:!!With income:!!$15,000 to $24,999       PLACE …
 6 B06010PR_007 Estimate!!Total:!!With income:!!$25,000 to $34,999       PLACE …
 7 B06010PR_008 Estimate!!Total:!!With income:!!$35,000 to $49,999       PLACE …
 8 B06010PR_009 Estimate!!Total:!!With income:!!$50,000 to $64,999       PLACE …
 9 B06010PR_010 Estimate!!Total:!!With income:!!$65,000 to $74,999       PLACE …
10 B06010PR_011 Estimate!!Total:!!With income:!!$75,000 or more          PLACE …
11 B06

In [13]:
%%R 
# the variable B01003_001E was selectd from the census table 
# for population, which we found in censusreporter here:
# https://censusreporter.org/tables/B01003/

# in the table below, pick the geography, the variables, and the survey you want to pull from
# see the possible values here https://walker-data.com/tidycensus/articles/basic-usage.html

# Get variable from ACS
census_data <- get_acs(geography = "zcta", 
                      variables = c(
                        population="B01003_001",
                        med_hh_inc="B19013_001",
                        med_fam_inc="B19113_001",
                        med_fam_inc_w_children="B19125_002",
                        gini_inequality="B19083_001",
                        poverty_pop="B17001_002",
                        white_pop ="B02001_002",
                        total_racial_pop="B02001_001",
                        total_edu_pop="B15003_001",
                        bach_pop="B15003_022",
                        masters_pop="B15003_023",
                        phd_pop="B15003_025",
                        prof_school_pop="B15003_024"
                      ), 
                      year = 2021,
                      survey="acs5",
                      geometry=T)

census_data

  |======================================================================| 100%
Simple feature collection with 439062 features and 5 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: -176.6967 ymin: 17.92667 xmax: -65.22108 ymax: 71.34122
Geodetic CRS:  NAD83
First 10 features:
   GEOID        NAME               variable   estimate        moe
 15301 ZCTA5 15301             population 49802.0000   788.0000
 15301 ZCTA5 15301             med_hh_inc 62902.0000  3704.0000
 15301 ZCTA5 15301            med_fam_inc 83360.0000  5018.0000
 15301 ZCTA5 15301 med_fam_inc_w_children 86179.0000 14806.0000
 15301 ZCTA5 15301        gini_inequality     0.4349     0.0184
 15301 ZCTA5 15301            poverty_pop  4838.0000   602.0000
 15301 ZCTA5 15301              white_pop 44482.0000   978.0000
 15301 ZCTA5 15301       total_racial_pop 49802.0000   788.0000
 15301 ZCTA5 15301          total_edu_pop 36138.0000   829.0000
 15301 ZCTA5 15301               bach_pop  6081.0000   

Getting data from the 2017-2021 5-year ACS


In [15]:
%%R 

pivoted_census <- census_data %>%
    select(-moe) %>%
    pivot_wider(names_from = "variable", values_from="estimate") %>%
    mutate(poverty_prop = poverty_pop / population,
           white_prop = white_pop / total_racial_pop,
           bach_plus_prop = (bach_pop + masters_pop + phd_pop + prof_school_pop) / total_edu_pop)



In [16]:
%%R
library(stringr)

cleaned_census <- pivoted_census %>% mutate(zip_code = as.double(str_sub(NAME, -5,-1)))

## 👉 Merge it with your data

hint...`tidycensus` provides you data in long format you may need to pivot the census data from long to wide format before merging it with your data

In [17]:
%%R


school_census_data <- df %>% left_join(cleaned_census, by=join_by(LEA_ZIP == zip_code))

In [18]:
%%R 

write_csv(school_census_data, "../data/final_data.csv")